# Hn Stage to OMOP

In [ ]:
import mysql.connector
import pandas as pd
import psycopg2
import random
import numpy as np
from datetime import datetime, timedelta

In [ ]:
sarcomaStage_IDEA4RC = pd.read_csv("./IDEA4RC-data/sarcomaStageIDEA4RC.csv")
sarcomaStage_IDEA4RC.head(5)

In [ ]:
# Conexión a la base de datos OMOP CDM
conn = psycopg2.connect(
    dbname="omopdb",
    user="postgres",
    password="mysecretpassword",
    host="localhost",
    port="5432"
)

# Comando para hacer el mapeo (insercción de datos del IDEA4RC.csv a OMOP)
cur = conn.cursor()
config = {
    'user': 'user', 
    'password': 'password',
    'host': '127.0.0.1',
    'database': 'idea4rc_dm',
    'raise_on_warnings': True
}

conn2 = mysql.connector.connect(**config)
curIDEA = conn2.cursor()

### Sarcoma Stage to Measurement/Event

Regional nodal metastases
Soft tissue
distant lymph node
lung
bone
liver
pleura
peritoneum
brain
other viscera
unknown

These are BOOLEANS with VOCABULARY and variable as modifier, what should i do?

For Localised and Loco-regional, I believe I have to change the values in the table episode, since the episode should exist as the reference exists.

In [2]:
df_tables=sarcomaStage_IDEA4RC

cols= {
    'Localised' : 0,
    'Loco-regional' : 0,
    'Regional nodal metastases': 32944,
    'Soft tissue': 32945,
    'distant lymph node': 32946,
    'lung': 32947,
    'bone': 32948,
    'liver': 32949,
    'pleura': 32950,
    'peritoneum': 32951,
    'brain': 32952,
    'other viscera': 32952,
    'unknown': 32952
}
sql = """
    INSERT INTO omopcdm.measurement (person_id, measurement_concept_id, measurement_date,measurement_type_concept_id,value_as_concept_id,measurement_event_id)
    VALUES (%s, %s, %s,%s,%s,%s)
"""
query= """
    SELECT c.patient 
    FROM cancer_episode c
    WHERE EXISTS (
        SELECT 1 
        FROM episode_event e
        WHERE e.cancerEpisode = c.id
        AND e.id = %s
    )
    LIMIT 1

"""

measurement_type_concept_id=38000280
for idx, row in df_tables.iterrows():
    for column in cols.keys():
        if ((column=='Localised' and row[column]==32942)||(column=='Loco-regional' and row[column]==32943)):
            sql2="""
                UPDATE omopcdm.episode
                SET episode_concept_id = %s
                WHERE episode_id= %s;
            """
            cur.execute(sql2, (row[column],row['Episode Event Reference']))
            conn.commit()
        else:
            curIDEA.execute(query, (row['Episode Event Reference']))
            res=curIDEA.fetchone()
            person_id=res
            if(row[column] in boolValues=[4188539,4188540]):
                measurement_value = row[column]
            else:
                measurement_value = row[column]==0?4188540:4188539
            measurement_concept_id = cols.get(column)
            measurement_event_id=row['Episode Event Reference']
            cur.execute(sql, (person_id, measurement_concept_id, date, measurement_type_concept_id,measurement_value,measurement_event_id))
            conn.commit()



SyntaxError: invalid syntax (3276209882.py, line 36)